## Import Packages

In [1]:
import pandas as pd
import numpy as np
import requests
import json
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, Dropdown

In [2]:
URL = "https://dsci551-final-project-fd95f-default-rtdb.firebaseio.com/"
json_suffix = '.json'

## Data Preprocessing

### Data Loading and Remove Irrelevant Features

In [3]:
# crime data
# 2020-01-01~2021-08-16
crime_data = pd.read_csv("/Users/Charlie/Desktop/USC/課程/DSCI551/Project/data/Crime_Data_from_2020_to_Present.csv")
crime_data.drop(['Date Rptd', 'Rpt Dist No' , 'Crm Cd', 'Part 1-2', 'Mocodes', 'Crm Cd 1', 
                 'Crm Cd 2', 'Crm Cd 3', 'Crm Cd 4', 'Cross Street', 'LOCATION'], axis=1, inplace=True)

In [4]:
crime_data.head()

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Vict Descent,Premis Cd,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,2230,3,Southwest,BATTERY - SIMPLE ASSAULT,36,F,B,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,34.0141,-118.2978
1,190101086,01/01/2020 12:00:00 AM,330,1,Central,BATTERY - SIMPLE ASSAULT,25,M,H,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,34.0459,-118.2545
2,201220752,09/16/2020 12:00:00 AM,1230,12,77th Street,VANDALISM - MISDEAMEANOR ($399 OR UNDER),62,M,B,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,33.9739,-118.2630
3,191501505,01/01/2020 12:00:00 AM,1730,15,N Hollywood,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,415,19,Mission,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,X,409.0,BEAUTY SUPPLY STORE,NaN,NaN,IC,Invest Cont,34.2198,-118.4468


In [6]:
# covid_data
# 2020-03-10~2021-10-03
covid_data = pd.read_csv('/Users/Charlie/Desktop/USC/課程/DSCI551/Project/code/Final/LA_County_Covid19_tests_date_table.csv')
covid_data.drop(['Unnamed: 0'], axis=1, inplace=True)
covid_data = covid_data.dropna()
covid_data.head()

,date_use,cumulative_tests,cumulative_tests_pos,percent_positive_tests_cum,tests,tests_pos,percent_positive_tests,avg_tests,avg_pos_tests,percent_positive_avg_tests
0,2021/10/2,31283474,1740508,5.60%,814,29,3.60%,87437.0,1110.0,1.30%
1,2021/10/1,31282660,1740479,5.60%,67027,783,1.20%,91997.0,1265.0,1.40%
2,2021/9/30,31215633,1739696,5.60%,130159,1336,1%,106554.0,1396.0,1.30%
3,2021/9/29,31085474,1738360,5.60%,155981,1587,1%,116476.0,1443.0,1.20%
4,2021/9/28,30929493,1736773,5.60%,131566,1526,1.20%,120689.0,1482.0,1.20%


In [7]:
covid_data['percent_positive_tests_cum'] = covid_data['percent_positive_tests_cum'].str.rstrip('%').astype('float') / 100.0
covid_data['percent_positive_tests'] = covid_data['percent_positive_tests'].str.rstrip('%').astype('float') / 100.0
covid_data['percent_positive_avg_tests'] = covid_data['percent_positive_avg_tests'].str.rstrip('%').astype('float') / 100.0

In [8]:
covid_data.head()

,date_use,cumulative_tests,cumulative_tests_pos,percent_positive_tests_cum,tests,tests_pos,percent_positive_tests,avg_tests,avg_pos_tests,percent_positive_avg_tests
0,2021/10/2,31283474,1740508,0.056,814,29,0.036,87437.0,1110.0,0.013
1,2021/10/1,31282660,1740479,0.056,67027,783,0.012,91997.0,1265.0,0.014
2,2021/9/30,31215633,1739696,0.056,130159,1336,0.010,106554.0,1396.0,0.013
3,2021/9/29,31085474,1738360,0.056,155981,1587,0.010,116476.0,1443.0,0.012
4,2021/9/28,30929493,1736773,0.056,131566,1526,0.012,120689.0,1482.0,0.012


In [9]:
crime_data.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Crm Cd Desc',
       'Vict Age', 'Vict Sex', 'Vict Descent', 'Premis Cd', 'Premis Desc',
       'Weapon Used Cd', 'Weapon Desc', 'Status', 'Status Desc', 'LAT', 'LON'],
      dtype='object')

In [10]:
covid_data.columns

Index(['date_use', 'cumulative_tests', 'cumulative_tests_pos',
       'percent_positive_tests_cum', 'tests', 'tests_pos',
       'percent_positive_tests', 'avg_tests', 'avg_pos_tests',
       'percent_positive_avg_tests'],
      dtype='object')

### Data Transforming

In [11]:
# Convert to DataTime Format
crime_data['DATE OCC'] = pd.to_datetime(crime_data['DATE OCC'])
covid_data['date_use'] = pd.to_datetime(covid_data['date_use'])
# Rename Columns Name
crime_data.rename(columns={'DATE OCC': 'date','TIME OCC':'crime_time', 'AREA':'area', 'AREA NAME':'area_name',
                           'CRIME_TIME':'crime_time', 'LAT':'lat', 'LON':'lon','Vict Age':'vict_age',
                           'Vict Descent':'vict_des','Vict Sex':'vict_sex','Weapon Used Cd':'weapon'}, inplace=True)
covid_data.rename(columns={'date_use': 'date'}, inplace=True)

In [12]:
crime_data['year'] = crime_data['date'].dt.year
crime_data['month'] = crime_data['date'].dt.month
crime_data['day'] = crime_data['date'].dt.day
covid_data['year'] = covid_data['date'].dt.year
covid_data['month'] = covid_data['date'].dt.month
covid_data['day'] = covid_data['date'].dt.day

In [13]:
crime_data.head()

,DR_NO,date,crime_time,area,area_name,Crm Cd Desc,vict_age,vict_sex,vict_des,Premis Cd,Premis Desc,weapon,Weapon Desc,Status,Status Desc,lat,lon,year,month,day
0,10304468,2020-01-08,2230,3,Southwest,BATTERY - SIMPLE ASSAULT,36,F,B,501.0,SINGLE FAMILY DWELLING,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",AO,Adult Other,34.0141,-118.2978,2020,1,8
1,190101086,2020-01-01,330,1,Central,BATTERY - SIMPLE ASSAULT,25,M,H,102.0,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,34.0459,-118.2545,2020,1,1
2,201220752,2020-09-16,1230,12,77th Street,VANDALISM - MISDEAMEANOR ($399 OR UNDER),62,M,B,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,33.9739,-118.2630,2020,9,16
3,191501505,2020-01-01,1730,15,N Hollywood,VANDALISM - MISDEAMEANOR ($399 OR UNDER),76,F,W,502.0,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",NaN,NaN,IC,Invest Cont,34.1685,-118.4019,2020,1,1
4,191921269,2020-01-01,415,19,Mission,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",31,X,X,409.0,BEAUTY SUPPLY STORE,NaN,NaN,IC,Invest Cont,34.2198,-118.4468,2020,1,1


In [14]:
# Merge Data
#data = pd.merge(crime_data, covid_data, how='outer', on='DATE')
#data.sort_values(by='DATE', key=pd.to_datetime).reset_index(drop=True)

## Function Definition

In [15]:
def clean_database():
    response = requests.delete(URL + json_suffix)

In [16]:
def unique_date(dataframe):
    unique_date = dataframe['date'].unique()
    unique_date.sort()
    unique_dateList = list(dataframe['date'].unique())
    unique_dateList = np.datetime_as_string(unique_date, unit='D')
    return unique_dateList

In [17]:
def transform_to_json(df):
    json = df.to_json(orient = 'records')
    return json

In [18]:
def upload_data(url, index, year, month, date, data):
    try:
        database_URL = url + index + '/' + str(year) + '/' + str(month) + '/' + date  + json_suffix
        response = requests.put(database_URL, data)
    except:
        print("Upload Failed")

In [19]:
def upload_to_firebase(dataset, index):
    for year in dataset['year'].unique():
        df_year = dataset.loc[dataset['year']==year]
        for month in df_year['month'].unique():
            df_month = df_year.loc[df_year['month']==month]
            unique_dateList = unique_date(df_month)
            for i in unique_dateList:
                date = str(i)
                df = df_month.loc[df_month['date']==date]
                df.drop(['date'], axis=1)
                json_data = transform_to_json(df)
                upload_data(URL, index, year, month, date, json_data)

## Running

In [17]:
clean_database()

In [20]:
upload_to_firebase(crime_data ,'crime')

In [18]:
upload_to_firebase(covid_data ,'covid')

In [22]:
crime_data.to_csv('crime_data_modified.csv', index=False)

In [22]:
covid_data.to_csv('covid_modified.csv', index=False)